In [45]:
!pip install vectorbt
!pip install condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge ta-lib
!pip install vectorbt

  Using cached condacolab-0.1.9-py3-none-any.whl.metadata (5.6 kB)
Using cached condacolab-0.1.9-py3-none-any.whl (7.2 kB)
✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [1]:
import vectorbt as vbt
import talib as ta
import requests
import json
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import time
import datetime

In [38]:
class GetBinanceData:
    @staticmethod
    def get_data(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        dataList = []
        current_date = datetime.datetime.now()
        start_date = startDate

        delta_units = GetBinanceData.convert_to_minutes(interval)
        print(delta_units)
        delta = datetime.timedelta(minutes=delta_units)

        while start_date < current_date:
            end_date = start_date + delta
            if end_date > current_date:
                end_date = current_date
            # Convert datetime objects to milliseconds
            start_ms = int(start_date.timestamp() * 1000)
            end_ms = int(end_date.timestamp() * 1000)

            start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

            # print(f"Fetching data from {start_str} to {end_str}")
            data = GetBinanceData.get_historical_data(symbol, interval, start_ms, end_ms)
            if data:
                dataList.extend(data)
            start_date = end_date
            time.sleep(1)

        return dataList

    @staticmethod
    def get_historical_data(symbol, interval, start_time, end_time):
        url = f"https://api.binance.us/api/v3/klines"
        params = {
            'symbol': symbol.upper(),
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
        }
        # print(params)
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:

            print(f"Failed to fetch data. Status code: {response.json()}")
            return None

    @staticmethod
    def convert_to_minutes(time_str):
        time_mapping = {
            'm': 1,
            'h': 60,
            'd': 1440,
            'w': 10080,
            'M': 43200
        }

        num = int(time_str[:-1])
        unit = time_str[-1]

        return (num * time_mapping[unit])*1000

    def data_to_dataframe(data):
      # Define the column names based on Binance API response
      columns = [
          "date1", "open", "high", "low", "close", "volume",
          "date", "Quote Asset Volume", "Number of Trades",
          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
      ]
      # Convert the data to a pandas DataFrame
      df = pd.DataFrame(data, columns=columns)
      # # Convert timestamp columns to datetime
      # df["date"] = pd.to_datetime(df['date'])
      # df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')
      # Convert numeric columns to float
      numeric_columns = ["open", "high", "low", "close", "volume", "Quote Asset Volume",
                        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
      df[numeric_columns] = df[numeric_columns].astype(float)
      return df

    @staticmethod
    def get_data_dataFrame(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        data = GetBinanceData.get_data(symbol, interval, startDate, endDate)
        return GetBinanceData.data_to_dataframe(data)

In [4]:
timeperiod = '1h'

symbol = "btcusdt"
data = GetBinanceData.get_data(symbol, timeperiod, startDate=datetime.datetime(2022, 6, 1), endDate=datetime.datetime.now())


60000


In [39]:
dataframe = GetBinanceData.data_to_dataframe(data)  # 1-minute data


In [9]:
!pip install pandas_ta
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.2 MB/s eta 0:00:00


In [44]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import mplfinance as mpf

def vsa_indicator(data: pd.DataFrame, norm_lookback: int = 168) -> pd.Series:
    data['high'] = data['high'].astype(float)
    data['low'] = data['low'].astype(float)
    data['close'] = data['close'].astype(float)
    data['volume'] = data['volume'].astype(float)

    atr = ta.atr(data['high'], data['low'], data['close'], norm_lookback)
    vol_med = data['volume'].rolling(norm_lookback).median()

    data['norm_range'] = (data['high'] - data['low']) / atr
    data['norm_volume'] = data['volume'] / vol_med

    norm_vol = data['norm_volume'].to_numpy()
    norm_range = data['norm_range'].to_numpy()

    range_dev = np.zeros(len(data))
    range_dev[:] = np.nan

    for i in range(norm_lookback * 2, len(data)):
        window = data.iloc[i - norm_lookback + 1: i + 1]
        slope, intercept, r_val, _, _ = stats.linregress(window['norm_volume'], window['norm_range'])

        if slope <= 0.0 or r_val < 0.2:
            range_dev[i] = 0.0
            continue

        pred_range = intercept + slope * norm_vol[i]
        range_dev[i] = norm_range[i] - pred_range

    return pd.Series(range_dev, index=data.index)



dataframe['date'] = pd.to_datetime(dataframe['date'] / 1000, unit='s').astype('datetime64[s]')

dataframe = dataframe.set_index('date')

dataframe['dev'] = vsa_indicator(dataframe, 168)



KeyError: 'date'

In [42]:
print(dataframe.head())
# dataframe["date"].dtype

           date1      open      high       low     close    volume  \
0  1654041600000  31814.89  31978.77  31699.70  31942.30  5.860436   
1  1654045200000  31940.81  31940.81  31845.38  31920.34  2.778502   
2  1654048800000  31925.46  31928.45  31735.22  31808.84  6.386045   
3  1654052400000  31799.73  31799.73  31616.49  31649.21  1.895066   
4  1654056000000  31655.21  31670.43  31392.41  31563.39  6.718004   

            date  Quote Asset Volume  Number of Trades  \
0  1654045199999       186574.157257               500   
1  1654048799999        88611.179690               181   
2  1654052399999       203180.699796               242   
3  1654055999999        60028.741531               175   
4  1654059599999       211674.353591               519   

   Taker Buy Base Asset Volume  Taker Buy Quote Asset Volume Ignore  
0                     2.798294                  89081.033894      0  
1                     1.343767                  42866.463871      0  
2                   